In [1]:
import pandas as pd

file_path = '../Datasets/cleaned_dataset.csv'
df = pd.read_csv(file_path)

In [2]:
# Check the balance of the categories in the target variable
category_distribution = df['category'].value_counts(normalize=True)

category_distribution

category
biographies                             0.318044
philosophy                              0.279965
programming                             0.214409
artificial intelligence                 0.169624
movies about artificial intelligence    0.017958
Name: proportion, dtype: float64

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Preprocessing the 'has_entity' feature by encoding it as binary features
# Extracting whether the paragraph contains references to an organization, product, or person
df['has_org'] = df['has_entity'].apply(lambda x: 'ORG_YES' in x).astype(int)
df['has_product'] = df['has_entity'].apply(lambda x: 'PRODUCT_YES' in x).astype(int)
df['has_person'] = df['has_entity'].apply(lambda x: 'PERSON_YES' in x).astype(int)

# Vectorizing the 'paragraph' text with TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
X_text = tfidf.fit_transform(df['paragraph'])

# Encoding the categorical target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['category'])

# Combining the TF-IDF features with the 'has_entity' binary features
X_other_features = df[['has_org', 'has_product', 'has_person']].to_numpy()
X = np.hstack((X_text.toarray(), X_other_features))

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Output the shape of the training and testing sets to confirm the preprocessing steps
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7395, 5003), (1849, 5003), (7395,), (1849,))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Initialize the Logistic Regression model
lr_model = LogisticRegression(C=1, solver='saga', class_weight='balanced', random_state=42, max_iter=5000)

# Train the model
lr_model.fit(X_train, y_train)

# Predictions
y_pred = lr_model.predict(X_test)

# Evaluation
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)
print("\nAccuracy:", accuracy)
print("Macro F1-Score:", macro_f1)

# Cross-validation for stability check
cv_scores = cross_val_score(lr_model, X, y, cv=StratifiedKFold(5), scoring='f1_macro')
print("\nCV Macro F1-Scores:", cv_scores)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Initialize the Logistic Regression model with optimized parameters
lr_model = LogisticRegression(C=1, solver='saga', class_weight='balanced', random_state=42, max_iter=5000)

# Train the model
lr_model.fit(X_train, y_train)

# Predictions
y_pred = lr_model.predict(X_test)

# Evaluation
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)
print("\nAccuracy:", accuracy)
print("Macro F1-Score:", macro_f1)

# Cross-validation for stability check, with parallel processing
cv_scores = cross_val_score(lr_model, X, y, cv=StratifiedKFold(5), scoring='f1_macro', n_jobs=-1)
print("\nCV Macro F1-Scores:", cv_scores)